In [3]:
import numpy as np
import pandas as pd
import networkx as nx
from itertools import product
from ecomplexity import ecomplexity
from ecomplexity import proximity
from ecomplexity import calc_density
import country_converter as coco
import itertools

from utils import *
import warnings
warnings.filterwarnings('ignore')

In [5]:
files = ["languages.csv", "developers.csv"]
download_github_data(files)

languages.csv already exists
developers.csv already exists


**Part 1 - GitHub data preparation**

In [3]:
# parameter to choose year / semester / quarter to construct period IDs
selected_period = "year"

# for ecomplexity calculcation
key_cols = {
    "time": "period",
    "loc": "iso2_code",
    "prod": "language",
    "val": "num_pushers",
}

# main GitHub data IN
data = pd.read_csv("../data/languages.csv")

In [4]:
# use data_prep_functions to clean the dataframe of ECI_software calculation
data = data[data["year"].isin([2020, 2021, 2022, 2023])]
prev_filter = "|".join(["yaml", "json", "text", "svg", "Markdown", "xml"])
df = drop_specifics_from_list(data, filter_list=prev_filter)
df = top_languages_filter(df, nr_languages=150)
df = drop_country_codes_from_list(df, country_list=["EU"])
df = add_period_ids(df, period=selected_period)
print(df.shape)

(93076, 7)


**Part 2 - from M_cl to ECI_software and language proximity - based on yearly data**

In [5]:
# software complexity calculation -- period IDs -- 1 means 2020 on yearly basis
ccdf = []
ppdf = []
year_dict = {1 : 2020, 2 : 2021, 3 : 2022, 4 : 2023}
for k in year_dict.keys():
    dfb = bundle_data(df, periods=[k])
    cdf = ecomplexity(dfb, key_cols)
    cdf["year"] = year_dict[k]

    pdf = proximity(dfb, key_cols)
    pdf["year"] = year_dict[k]

    # combine yearly dataframes
    ccdf.append(cdf)
    ppdf.append(pdf)
    print(year_dict[k], " DONE")

1
Percentage of pairs compared that meet log-supermodularity condition: 2.67%
1
2020  DONE
1
Percentage of pairs compared that meet log-supermodularity condition: 4.96%
1
2021  DONE
1
Percentage of pairs compared that meet log-supermodularity condition: 5.28%
1
2022  DONE
1
Percentage of pairs compared that meet log-supermodularity condition: 4.93%
1
2023  DONE


In [6]:
# combine and save -- complexity
cdf = pd.concat(ccdf, axis=0, ignore_index=True)
cdf.to_csv("../outputs/eci_software_2020_2023.csv", sep=";", index=False)

In [7]:
# combine and save -- language proximity
prox_df = pd.concat(ppdf, axis=0, ignore_index=True)
prox_df.to_csv("../outputs/proximity_2020_2023.csv", sep=";", index=False)

**Part 3 - ECI(software, trade, technology, research) and macroecon indicators**

In [8]:
# add ECI_software value
eci_software = pd.read_csv("../outputs/eci_software_2020_2023.csv", sep=";")

# country code switcher
iso2_codes = eci_software["iso2_code"]
iso3_codes = coco.convert(names=iso2_codes, to="ISO3")
codes = pd.DataFrame(iso3_codes, iso2_codes).reset_index()
codes.columns = ["iso2_code", "iso3_code"]
codes = dict(zip(codes["iso2_code"], codes["iso3_code"]))

# clean table for 2020
eci_software["iso3_code"] = eci_software["iso2_code"].map(codes)
eci_software = eci_software[["iso2_code", "iso3_code", "eci", "year"]].drop_duplicates()
eci_software.rename(columns={"eci":"eci_software"}, inplace=True)

In [9]:
# read in 3 other ECI measures from OEC
eci_trade = pd.read_csv("../data/Data-ECI-Trade.csv")
eci_trade = eci_trade[["Country", "2020", "2021", "2022"]]
eci_trade = pd.melt(
        eci_trade,
        id_vars=["Country"],
        value_vars=["2020", "2021", "2022"],
        var_name="year",
        value_name="eci_trade",
    ).rename(columns={"Country":"country_name"})
eci_tech = pd.read_csv("../data/Data-ECI-Technology.csv")
eci_tech = eci_tech[["Country", "2020", "2021", "2022"]]
eci_tech = pd.melt(
        eci_tech,
        id_vars=["Country"],
        value_vars=["2020", "2021", "2022"],
        var_name="year",
        value_name="eci_tech",
    ).rename(columns={"Country":"country_name"})
eci_research = pd.read_csv("../data/Data-ECI-Research.csv")
eci_research = eci_research[["Country", "2020", "2021", "2022"]]
eci_research = pd.melt(
        eci_research,
        id_vars=["Country"],
        value_vars=["2020", "2021", "2022"],
        var_name="year",
        value_name="eci_research",
    ).rename(columns={"Country":"country_name"})

In [10]:
# map country names to iso2_codes - using the country_converter package
c_to_iso = dict(
    zip(eci_trade.country_name.unique(), coco.convert(names=eci_trade.country_name.unique(), to="ISO2")))
eci_trade["iso2_code"] = eci_trade["country_name"].map(c_to_iso)
c_to_iso = dict(
    zip(eci_tech.country_name.unique(), coco.convert(names=eci_tech.country_name.unique(), to="ISO2")))
eci_tech["iso2_code"] = eci_tech["country_name"].map(c_to_iso)
c_to_iso = dict(
    zip(eci_research.country_name.unique(), coco.convert(names=eci_research.country_name.unique(), to="ISO2")))
eci_research["iso2_code"] = eci_research["country_name"].map(c_to_iso)

In [11]:
# join ECI tables
eci_software["year"] = eci_software["year"].astype(int)
eci_trade["year"] = eci_trade["year"].astype(int)
eci_tech["year"] = eci_tech["year"].astype(int)
eci_research["year"] = eci_research["year"].astype(int)

eci_df = pd.merge(
    eci_software,
    eci_trade,
    on=["iso2_code", "year"]
)
eci_df = pd.merge(
    eci_df,
    eci_tech,
    on=["iso2_code", "year"]
)
eci_df = pd.merge(
    eci_df,
    eci_research,
    on=["iso2_code", "year"]
).drop(columns=["country_name_x", "country_name_y"])

In [12]:
# World Bank -- general indicators
wdf = pd.read_csv("../data/worldbank_general_indicators_2018_2023.csv")\
    .rename(columns={"Series Name":"variable", "Country Name":"country_name", "Country Code":"iso3_code"})

# variable names
names_list = ["Population, total", "GDP per capita (current US$)", "GDP growth (annual %)", "GDP (current US$)"]
new_names = ["population", "gdp_per_capita", "gdp_growth", "gdp_current_USD"]

country_df_general = world_bank_data_cleaner(wdf, names_list=names_list, new_names=new_names)

In [13]:
# World Bank -- GDP PPP
wdf_gdp_ppp = pd.read_excel("../data/worldbank_gdp_ppp_1960_2023.xls")\
    .rename(columns={"Indicator Name":"variable", "Country Name":"country_name", "Country Code":"iso3_code"})

# variable names
gdp_ppp_df = pd.melt(
        wdf_gdp_ppp,
        id_vars=["variable", "country_name", "iso3_code"],
        value_vars=[str(year) for year in range(2010, 2023)],
        var_name="year",
        value_name="gdp_ppp",
    ).drop(columns=["variable"])
gdp_ppp_df["year"] = gdp_ppp_df["year"].astype(int)

In [14]:
# World Bank data on natural resources
wdf_nat = pd.read_csv("../data/worldbank_total_natural_resources_rents_GPD_perc_2014_2021.csv")\
    .rename(columns={"Series Name":"variable", "Country Name":"country_name", "Country Code":"iso3_code"})

# variable names
names_list = ["Total natural resources rents (% of GDP)"]
new_names = ["natural_resources"]

country_df_natural_res = world_bank_data_cleaner(wdf_nat, names_list=names_list, new_names=new_names)

In [15]:
# World Bank data on human capital
wdf_hum = pd.read_csv("../data/worldbank_human_capital_indicators_2018_2020.csv")\
    .rename(columns={"Series Name":"variable", "Country Name":"country_name", "Country Code":"iso3_code"})

# variable names
names_list = ["Human Capital Index (HCI) (scale 0-1)"]
new_names = ["human_capital_index"]

country_df_human_cap = world_bank_data_cleaner(wdf_hum, names_list=names_list, new_names=new_names)

In [16]:
# World Bank data on Gini -- https://data.worldbank.org/indicator/SI.POV.GINI
wdf_gini = pd.read_excel("../data/worldbank_gini.xls")\
    .rename(columns={"Indicator Name":"variable", "Country Name":"country_name", "Country Code":"iso3_code"})

# variable names
gini_df = pd.melt(
        wdf_gini,
        id_vars=["variable", "country_name", "iso3_code"],
        value_vars=[str(year) for year in range(2010, 2023)],
        var_name="year",
        value_name="gini",
    ).drop(columns=["variable"])

# mean Gini -- too many missing data points
gini_df["gini_mean"] = gini_df.groupby(["country_name", "iso3_code"])["gini"].transform("mean")
gini_df["year"] = gini_df["year"].astype(int)
gini_df["gini_mean_2020_2022"] = gini_df[gini_df["year"]>=2020].groupby(["country_name", "iso3_code"])["gini"].transform("mean")

In [4]:
# World Bank data on emissions
wdf_emission = pd.read_excel("../data/worldbank_emission_1960_2022.xlsx")
#wdf_emission = wdf_emission[wdf_emission["Indicator"].str.contains("Total greenhouse gas emissions (kt of CO2 equivalent)", na=False)]\
wdf_emission = wdf_emission[wdf_emission["Indicator"] == "Total greenhouse gas emissions (kt of CO2 equivalent)"]\
    .rename(columns={"Indicator":"variable", "Economy Name":"country_name", "Economy ISO3":"iso3_code"})

# variable names
emission_df = pd.melt(
        wdf_emission,
        id_vars=["variable", "iso3_code"],
        value_vars=[str(year) for year in range(2010, 2022)],
        var_name="year",
        value_name="total_ghg_emissions",
    ).drop(columns=["variable"])
emission_df["year"] = emission_df["year"].astype(int)

In [18]:
# join emission data from Viktor
emissions_viktor = pd.read_csv("../data/regressions_emissions_data.csv")
emissions_viktor = emissions_viktor[["country", "year", "emissions"]].drop_duplicates().rename(columns={"country":"iso3_code", "emissions":"emission_viktor"})
emissions_viktor["year"] = emissions_viktor["year"].astype(int)

In [19]:
# join World Bank tables
rdf = pd.merge(
    country_df_general,
    gdp_ppp_df,
    on=["country_name", "iso3_code", "year"],
    how="left"
)
rdf = pd.merge(
    rdf,
    country_df_natural_res,
    on=["country_name", "iso3_code", "year"],
    how="left"
)
rdf = pd.merge(
    rdf,
    country_df_human_cap,
    on=["country_name", "iso3_code", "year"],
    how="left"
)
rdf = pd.merge(
    rdf,
    gini_df,
    on=["country_name", "iso3_code", "year"],
    how="left"
)

# mean Gini correction
rdf["gini_mean"] = rdf.groupby(["country_name", "iso3_code"])["gini_mean"].transform(lambda x: x.fillna(x.mean()))

# add emissions
rdf["year"] = rdf["year"].astype(int)
rdf = pd.merge(
    rdf,
    emission_df,
    on=["iso3_code", "year"],
    how="left"
)
rdf = pd.merge(
    rdf,
    emissions_viktor,
    on=["iso3_code", "year"],
    how="left"
)

# replace .. w/ NA
rdf.replace([".."], np.nan, inplace=True)

In [20]:
# join ECI and country information
reg_df = pd.merge(
    eci_df,
    rdf,
    on=["iso3_code", "year"],
    how="outer",
    suffixes=["", "2"]
).drop(columns="country_name2")

In [21]:
# export for regressions in R
reg_df.to_csv("../outputs/eci_regression_table.csv", sep=";", index=False)

**Part 2b - matrices based on trade/research/publications**

In [7]:
# read and reshape matrices
trade_df = mat_reshape(path="../data/stojkoski_etal_data/trade_matrix_data_2020.csv", column_labels=["product", "iso3_code", "value"])
patent_df = mat_reshape(path="../data/stojkoski_etal_data/pct_data_2020.csv", column_labels=["class", "iso3_code", "value"])
research_df = mat_reshape(path="../data/stojkoski_etal_data/pub_matrix_data_2020.csv", column_labels=["category", "iso3_code", "value"])


In [8]:
# read and reshape matrices
trade_df = mat_reshape(path="../data/stojkoski_etal_data/trade_matrix_data_2020.csv", column_labels=["product", "iso3_code", "value"])
patent_df = mat_reshape(path="../data/stojkoski_etal_data/pct_data_2020.csv", column_labels=["class", "iso3_code", "value"])
research_df = mat_reshape(path="../data/stojkoski_etal_data/pub_matrix_data_2020.csv", column_labels=["category", "iso3_code", "value"])

# country code correction
iso3_codes = pd.concat([trade_df["iso3_code"], patent_df["iso3_code"], research_df["iso3_code"]]).unique().tolist()
iso2_codes = coco.convert(names=iso3_codes, to="ISO2")
codes2 = pd.DataFrame(iso3_codes, iso2_codes).reset_index()
codes2.columns = ["iso2_code", "iso3_code"]

trade_df = pd.merge(
    trade_df,
    codes2,
    on="iso3_code",
    how="left"
)
trade_df = trade_df[trade_df["iso2_code"] != "not found"]

patent_df = pd.merge(
    patent_df,
    codes2,
    on="iso3_code",
    how="left"
)
patent_df = patent_df[patent_df["iso2_code"] != "not found"]

research_df = pd.merge(
    research_df,
    codes2,
    on="iso3_code",
    how="left"
)
research_df = research_df[research_df["iso2_code"] != "not found"]

ANT not found in ISO3
YUG not found in ISO3
CSE not found in ISO3
DDE not found in ISO3
EPO not found in ISO3
XKO not found in ISO3
SFE not found in ISO3
SUE not found in ISO3
XTP not found in ISO3
XUB not found in ISO3
FST not found in ISO3
PIT not found in ISO3


In [9]:
# World Bank -- general indicators
wdf = pd.read_csv("../data/worldbank_general_indicators_2018_2023.csv")\
    .rename(columns={"Series Name":"variable", "Country Name":"country_name", "Country Code":"iso3_code"})

# variable names
names_list = ["Population, total", "GDP per capita (current US$)", "GDP growth (annual %)", "GDP (current US$)"]
new_names = ["population", "gdp_per_capita", "gdp_growth", "gdp_current_USD"]

country_df_general = world_bank_data_cleaner(wdf, names_list=names_list, new_names=new_names)

# population above 1 million
country_df_general["population"] = pd.to_numeric(country_df_general["population"], errors="coerce").round().astype("Int64")
countries_1m_pop = list(set(country_df_general[country_df_general["population"]>1000000]["iso3_code"].to_list()))
trade_df = trade_df[trade_df["iso3_code"].isin(countries_1m_pop)]
patent_df = patent_df[patent_df["iso3_code"].isin(countries_1m_pop)]
research_df = research_df[research_df["iso3_code"].isin(countries_1m_pop)]

# total export value of 1 billion USD
above_1b_export = trade_df.groupby(["iso2_code"])["value"].agg("sum").reset_index()
above_1b_export = list(set(above_1b_export[above_1b_export["value"]>10**9]["iso2_code"].to_list()))
trade_df = trade_df[trade_df["iso2_code"].isin(above_1b_export)]
print(trade_df.shape)

# MIN 4 patent
min4_patents = patent_df.groupby(["iso2_code"])["value"].agg("sum").reset_index()
min4_patents = list(set(min4_patents[min4_patents["value"] > 4]["iso2_code"].to_list()))
patent_df = patent_df[patent_df["iso2_code"].isin(min4_patents)]
print(patent_df.shape)

# countries w/ MIN 100 publications in a year - category w/ more than 30 published papers a year
min100_publications = research_df.groupby(["iso2_code"])["value"].agg("sum").reset_index()
min100_publications = list(set(min100_publications[min100_publications["value"] >= 100]["iso2_code"].to_list()))
min30_papers = research_df.groupby(["category"])["value"].agg("sum").reset_index()
min30_papers = list(set(min30_papers[min30_papers["value"] >= 30]["category"].to_list()))
research_df = research_df[(research_df["category"].isin(min30_papers))]

# replace below 3 papers per country/category to 0
research_df["value"] = np.where(research_df["value"]<3, 0, research_df["value"])
print(research_df.shape)

(178788, 4)
(50768, 4)
(49131, 4)


In [10]:
# replace below avg 100 citations per country/category to 0
years = [2017, 2018, 2019, 2020]
citations = []
for y in years:
    temp = mat_reshape(path=f"../data/stojkoski_etal_data/cit_matrix_data_{y}.csv", column_labels=["category", "iso3_code", "citations"])
    temp["year"] = y
    citations.append(temp)

citations = pd.concat(citations)
citations = citations.groupby(["category", "iso3_code"])["citations"].agg("mean").reset_index()

research_df = pd.merge(
    research_df,
    citations,
    on=["iso3_code", "category"],
    how="left"
)
research_df["value"] = np.where(research_df["citations"]<100, 0, research_df["value"])

In [11]:
# calculate complexity and mcp
key_cols_trade = {
    "time": "period",
    "loc": "iso2_code",
    "prod": "product",
    "val": "value",
}
trade_df["period"] = 1
trade_cdf = ecomplexity(trade_df, key_cols_trade)

key_cols_patent = {
    "time": "period",
    "loc": "iso2_code",
    "prod": "class",
    "val": "value",
}
patent_df["period"] = 1
patent_cdf = ecomplexity(patent_df, key_cols_patent)

key_cols_research = {
    "time": "period",
    "loc": "iso2_code",
    "prod": "category",
    "val": "value",
}
research_df["period"] = 1
research_cdf = ecomplexity(research_df, key_cols_research)

1
Percentage of pairs compared that meet log-supermodularity condition: 38.67%
1
Percentage of pairs compared that meet log-supermodularity condition: 3.64%
1
Percentage of pairs compared that meet log-supermodularity condition: 14.47%


In [12]:
# save for figures
trade_cdf.to_csv("../outputs/trade_cdf_2020.csv", sep=";", index=False)
patent_cdf.to_csv("../outputs/patent_cdf_2020.csv", sep=";", index=False)
research_cdf.to_csv("../outputs/research_cdf_2020.csv", sep=";", index=False)

**Part 3 - for entry regressions**

In [27]:
# relatedness density -- as in Hidalgo et al. (2007) Science
cdf = pd.read_csv("../outputs/eci_software_2020_2023.csv", sep=";")
rel_dens = cdf[cdf["year"] == 2020][["iso2_code", "language", "density"]].drop_duplicates()

In [28]:
# data IN
data = pd.read_csv("../data/languages.csv")
selected_period = "year"

# steps to prep dataframe of ecomplexity
prev_filter = "|".join(["yaml", "json", "text", "svg", "Markdown", "xml"])
df = drop_specifics_from_list(data, filter_list=prev_filter)
df = top_languages_filter(df, nr_languages=150)
df = drop_country_codes_from_list(df, country_list=["EU"])
df = add_period_ids(df, period=selected_period)
print(df.shape)

(93076, 7)


In [29]:
# threshold for RCA : 1
ps = [1, 2, 3, 4]
rca_tables = []
for p in ps:
    print(p)
    temp = bundle_data(df, periods=[p])
    temp["period"] = p
    rca_tables.append(rca_calculation(temp, c_column="iso2_code", p_column="language", value_column="num_pushers", threshold=1))

#    dfbs.append(temp)
rca_tables = pd.concat(rca_tables)

1
2
3
4


In [30]:
# identify entry following the given patterns
entry_pattern = [0,0,1,1]
consider_pattern = [0,0,0,0]
ent = rca_tables.sort_values(["period"], ascending=True).groupby(["iso2_code","language"])["rca01"].agg(list).reset_index()
ent["entry01"] = ent["rca01"].apply(lambda x: x == entry_pattern).astype(int)
ent["consider00"] = ent["rca01"].apply(lambda x: x == consider_pattern).astype(int)

In [31]:
# full combination
all_countries = ent["iso2_code"].unique()
all_languages = ent["language"].unique()

all_combinations = list(product(all_countries, all_languages))
full_df = pd.DataFrame(all_combinations, columns=["iso2_code", "language"])\
    .sort_values(["iso2_code", "language"])

# join entries
full_df = pd.merge(
    full_df,
    ent[["iso2_code", "language", "entry01", "consider00"]],
    on=["iso2_code", "language"],
    how="left"
).fillna(0)

# join complexity
cdf = pd.read_csv("../outputs/eci_software_2020_2023.csv", sep=";")
cdf = cdf[cdf["year"]==2020]
full_df = pd.merge(
    full_df,
    cdf[["iso2_code", "language", "pci", "ubiquity"]],
    on=["iso2_code", "language"],
    how="left"
)

# join RCA from the baseline period
full_df = pd.merge(
    full_df,
    rca_tables[rca_tables["period"]==3].loc[:,["iso2_code", "language", "rca01"]],
    on=["iso2_code", "language"],
    how="left"
)
full_df["rca01"] = full_df["rca01"].fillna(0)

# drop languages with no complexity value
full_df.dropna(subset=["pci"], inplace=True)

In [32]:
# join to full_df with entries and PCI
full_df = pd.merge(
    full_df,
    rel_dens,
    on=["iso2_code", "language"],
    how="left"
)

In [33]:
# export for entry models -- only consider 00, 01 patterns
full_df["entry01"] = full_df["entry01"].astype(int)
full_df["consider00"] = full_df["consider00"].astype(int)
export_df = full_df[(full_df["entry01"]==1) | (full_df["consider00"]==1)]
export_df.to_csv("../outputs/data_entry_regressions_0011.csv", index=False, sep=";")
#export_df.to_csv("../outputs/data_entry_regressions_0011_threshold05.csv", index=False, sep=";")

**Part 4 - for exit regressions**

In [34]:
# relatedness density -- as in Hidalgo et al. (2007) Science
cdf = pd.read_csv("../outputs/eci_software_2020_2023.csv", sep=";")
rel_dens = cdf[cdf["year"] == 2020][["iso2_code", "language", "density"]].drop_duplicates()

In [35]:
# data IN
data = pd.read_csv("../data/languages.csv")
selected_period = "year"

# steps to prep dataframe of ecomplexity
prev_filter = "|".join(["yaml", "json", "text", "svg", "Markdown", "xml"])
df = drop_specifics_from_list(data, filter_list=prev_filter)
df = top_languages_filter(df, nr_languages=150)
df = drop_country_codes_from_list(df, country_list=["EU"])
df = add_period_ids(df, period=selected_period)
print(df.shape)

(93076, 7)


In [36]:
# threshold for RCA : 1
ps = [1, 2, 3, 4]
rca_tables = []
for p in ps:
    print(p)
    temp = bundle_data(df, periods=[p])
    temp["period"] = p
    rca_tables.append(rca_calculation(temp, c_column="iso2_code", p_column="language", value_column="num_pushers", threshold=1))

#    dfbs.append(temp)
rca_tables = pd.concat(rca_tables)

1
2
3
4


In [37]:
# identify entry following the given patterns
exit_pattern = [1,1,0,0]
consider_pattern = [1,1,1,1]
ext = rca_tables.sort_values(["period"], ascending=True).groupby(["iso2_code","language"])["rca01"].agg(list).reset_index()
ext["entry01"] = ext["rca01"].apply(lambda x: x == exit_pattern).astype(int)
ext["consider00"] = ext["rca01"].apply(lambda x: x == consider_pattern).astype(int)

In [38]:
# full combination
all_countries = ext["iso2_code"].unique()
all_languages = ext["language"].unique()

all_combinations = list(product(all_countries, all_languages))
full_df = pd.DataFrame(all_combinations, columns=["iso2_code", "language"])\
    .sort_values(["iso2_code", "language"])

# join entries
full_df = pd.merge(
    full_df,
    ext[["iso2_code", "language", "entry01", "consider00"]],
    on=["iso2_code", "language"],
    how="left"
).fillna(0)

# join complexity
cdf = pd.read_csv("../outputs/eci_software_2020_2023.csv", sep=";")
cdf = cdf[cdf["year"]==2020]
full_df = pd.merge(
    full_df,
    cdf[["iso2_code", "language", "pci", "ubiquity"]],
    on=["iso2_code", "language"],
    how="left"
)

# join RCA from the baseline period
full_df = pd.merge(
    full_df,
    rca_tables[rca_tables["period"]==3].loc[:,["iso2_code", "language", "rca01"]],
    on=["iso2_code", "language"],
    how="left"
)
full_df["rca01"] = full_df["rca01"].fillna(0)

# drop languages with no complexity value
full_df.dropna(subset=["pci"], inplace=True)

In [39]:
# join to full_df with entries and PCI
full_df = pd.merge(
    full_df,
    rel_dens,
    on=["iso2_code", "language"],
    how="left"
)

In [40]:
# export for entry models -- only consider 00, 01 patterns
full_df["entry01"] = full_df["entry01"].astype(int)
full_df.rename(columns={"entry01":"exit01"}, inplace=True)
full_df["consider00"] = full_df["consider00"].astype(int)
export_df = full_df[(full_df["exit01"]==1) | (full_df["consider00"]==1)]
export_df.to_csv("../outputs/data_exit_regressions_1100.csv", index=False, sep=";")
#export_df.to_csv("../outputs/data_entry_regressions_0011_threshold05.csv", index=False, sep=";")

**IV -- 3 most similar non-neighboring countries**

In [41]:
# ECI_software table
cdf = pd.read_csv("../outputs/eci_software_2020_2023.csv", sep=";")

# neighboring countries from https://github.com/geodatasource/country-borders
nc = pd.read_csv("../data/geodatasource_country_borders.csv")

In [42]:
locations = list(set(cdf["iso2_code"].to_list()))
full_prod_countries = pd.DataFrame(itertools.product(locations, repeat=2), columns=["iso2_code1", "iso2_code2"])


In [43]:
full_prod_countries = pd.merge(
    full_prod_countries,
    nc,
    left_on=["iso2_code1", "iso2_code2"],
    right_on=["country_code", "country_border_code"],
    how="left"
)
full_prod_countries["neighbor01"] = full_prod_countries["country_border_code"].notna().astype(int)
full_prod_countries = full_prod_countries[["iso2_code1", "iso2_code2", "neighbor01"]]

In [44]:
# select year
year_list = [2020, 2021, 2022, 2023]
cdf2 = []
for y in year_list:
    print(y)

    tcdf = cdf[cdf["year"] == y]
    tcdf.year.isna().sum()

    # generate full product dataframe
    locations = list(set(tcdf["iso2_code"].to_list()))
    full_prod_countries = pd.DataFrame(itertools.product(locations, repeat=2), columns=["iso2_code1", "iso2_code2"])
    full_prod_countries = pd.merge(
        full_prod_countries,
        nc,
        left_on=["iso2_code1", "iso2_code2"],
        right_on=["country_code", "country_border_code"],
        how="left"
    )
    full_prod_countries["neighbor01"] = full_prod_countries["country_border_code"].notna().astype(int)
    full_prod_countries = full_prod_countries[["iso2_code1", "iso2_code2", "neighbor01"]]

    # add location - mcp array to location pairs
    mcp_temp = tcdf.groupby("iso2_code")["mcp"].apply(np.array).reset_index()
    full_prod_countries = pd.merge(
        full_prod_countries,
        mcp_temp,
        left_on="iso2_code1",
        right_on="iso2_code",
        how="left"
    )
    full_prod_countries = pd.merge(
        full_prod_countries,
        mcp_temp,
        left_on="iso2_code2",
        right_on="iso2_code",
        how="left"
    )
    full_prod_countries = full_prod_countries\
        .drop(columns=["iso2_code_x", "iso2_code_y"])\
        .rename(columns={"mcp_x":"mcp_array1", "mcp_y":"mcp_array2"})

    # minimum conditional probability -- to measure similarity between locations
    full_prod_countries["spec_similarity"] = full_prod_countries.apply(lambda r: round(sum(r["mcp_array1"] * r["mcp_array2"]) / max(sum(r["mcp_array1"]), sum(r["mcp_array2"])), 3), axis=1)

    # drop iso2_code1 == iso2_code2 cases and neighbors
    sim_spec_df = full_prod_countries[(full_prod_countries["iso2_code1"] != full_prod_countries["iso2_code2"]) & (full_prod_countries["neighbor01"] == 0)]
    
    # keep the top3 most similar countries
    sim_spec_df = sim_spec_df.groupby(["iso2_code1"])["spec_similarity"]\
        .nlargest(3)\
        .reset_index()\
        .rename(columns={"level_1":"iso2_code2_index"})

    # merge similar location names by index
    sim_spec_df = pd.merge(
        sim_spec_df,
        full_prod_countries[["iso2_code2"]].reset_index(),
        left_on="iso2_code2_index",
        right_on="index",
        how="left"
    )

    # merge ECI values by location name
    sim_spec_df = pd.merge(
        sim_spec_df,
        tcdf[["iso2_code", "eci"]].drop_duplicates(),
        left_on="iso2_code2",
        right_on="iso2_code",
        how="left"
    )

    # merge distance values by location name
    sim_spec_df = pd.merge(
        sim_spec_df,
        full_prod_countries,
        on=["iso2_code1", "iso2_code2"],
        how="left"
    )

    # average ECI of the top 3 most similar location 
    avg_comp_sim_spec = sim_spec_df.groupby(["iso2_code1"])\
        .agg(
            avg_eci_similar_spec = pd.NamedAgg("eci", np.mean))\
        .reset_index()\
        .rename(columns={"iso2_code1" : "iso2_code"})

    # join to full comb table
    tcdf = pd.merge(
        tcdf,
        avg_comp_sim_spec,
        on="iso2_code",
        how="left"
    )
    cdf2.append(tcdf)

2020
2021
2022
2023


In [45]:
# join and save
cdf2 = pd.concat(cdf2)
cdf2.to_csv(f"../outputs/si_eci_software_2020_2023_ivreg.csv", index=False, sep=";")